In [1]:
import requests

In [4]:
res = requests.get("http://localhost:8949/nginx_status")
text = res.text

In [5]:
lines = text.strip().splitlines()

In [6]:
print(lines)

['Active connections: 2 ', 'server accepts handled requests', ' 52 52 254 ', 'Reading: 0 Writing: 1 Waiting: 1']


In [8]:
ac = int(lines[0].split(":")[1].strip())

In [9]:
print(ac)

2


In [10]:
nums = lines[2].strip().split()
accepts, handled, req_count = map(int, nums)

In [11]:
print(nums)

['52', '52', '254']


In [12]:
parts = lines[3].split()
reading = int(parts[1])
writing = int(parts[3])
waiting = int(parts[5])

In [13]:
print("ac =", ac)
print("accepts =", accepts)
print("handled =", handled)
print("requests =", req_count)
print("reading =", reading)
print("writing =", writing)
print("waiting =", waiting)

ac = 2
accepts = 52
handled = 52
requests = 254
reading = 0
writing = 1
waiting = 1


In [ ]:
import requests
import time
import csv
import os

filename = 'nginx_status_log.csv'

# 파일이 없으면 헤더를 작성합니다.
if not os.path.exists(filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['timestamp', 'active_connections', 'delta_accepts', 'delta_handled', 'delta_requests', 'reading', 'writing', 'waiting'])

prev_accepts = prev_handled = prev_requests = None

while True:
    try:
        # nginx 상태 페이지에서 데이터 가져오기
        res = requests.get("http://127.0.0.1:8949/nginx_status")
        text = res.text.strip()
        lines = text.splitlines()

        # 첫 번째 줄: Active connections 값 추출
        ac = int(lines[0].split(":")[1].strip())

        # 세 번째 줄: 누적값(accepts, handled, requests) 추출
        nums = lines[2].strip().split()
        accepts, handled, req_count = map(int, nums)

        # 네 번째 줄: Reading, Writing, Waiting 값 추출
        parts = lines[3].split()
        reading = int(parts[1])
        writing = int(parts[3])
        waiting = int(parts[5])

        # 누적값의 구간별(Δ) 변화 계산 (첫 측정 시 0으로 처리)
        if prev_accepts is not None:
            delta_accepts = accepts - prev_accepts
            delta_handled = handled - prev_handled
            delta_requests = req_count - prev_requests
        else:
            delta_accepts = delta_handled = delta_requests = 0

        # 현재 타임스탬프 기록
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')

        # CSV 파일에 데이터 append
        with open(filename, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([timestamp, ac, delta_accepts, delta_handled, delta_requests, reading, writing, waiting])

        # 이전 누적값 업데이트
        prev_accepts, prev_handled, prev_requests = accepts, handled, req_count

        # 10초 대기
        time.sleep(10)
    except Exception as e:
        print("Error occurred:", e)
        time.sleep(10)